# Pangenomics
--------------------------------------------

# Indexing Graphs with vg

## Overview

The Variation Graph Toolkit (VG) allows us to perform different operations on pangenomic graphs. You will learn about VG's capabilities and use VG to index PGGB graphs and get some stats.

## Learning Objectives
+ Understand VG and the different things it can be used for
+ Learn how to index graphs 
+ Get stats on the graphs

## Get Started

We will use the Variation Graph Toolkit (VG) to index our PGGB graphs, map sequences to them, and call variants.

However, VG can also create graphs and do many other steps in pangenomic analysis.
If you would like to learn about how to construct and manipulate graphs using VG and other pangenomic tools, please see [our virtual workshops](https://inbre.ncgr.org/ncgr-workshops/upcoming-ncgr-workshops.html).


### Variation Graph Toolkit (VG)

While we will not use VG to create pangenomics graphs in this module, it is important to understand the kinds of graphs that VG understands.

VG creates graphs that are cyclic, meaning that paths through the graph can be revisited.
This is important for capturing, for example, duplicated genomic regions.

VG graphs are otherwise general.
They are considered reference graphs, iterative, and reference-free.

VG has tools that can do the following pangenomic steps.

+ Constructs graphs
+ Manipulates graphs
+ Indexes graphs
+ Maps sequences to graphs
+ Calls variants on mapped sequences
+ Visualizes graphs

VG can also do:

+ [Transcriptomic analysis](https://github.com/vgteam/vg#transcriptomic-analysis)
+ Assembly-based pipelines
+ So much more

Citation:

![Garrison, E., Sirén, J., Novak, A. et al.](./Figures/VGref.png)

![vg Graph Genomics Pipeline: https://github.com/vgteam/vg](./Figures/VGpipe.png)


A reference genome "decorated" with variants:

![GRAF™ Pan Genome Reference: https://www.sevenbridges.com/graf/](./Figures/GRAF.png)
 

### Converting our graphs from GFA to VG format

Previously, you created these graphs using PGGB:  
yprp.chrVIII.pggb.gfa (a graph for yeast chromosome VIII)  
output_allchrs/*/*gfa (16 subgraphs representing the 16 yeast chromosomes)  
yprp.yprp.fullgenome.pggb.gfa.pggb.gfa (a graph for the entire yeast genome)

Before indexing, we need to convert the graphs from GFA to VG format.

NOTE: You can index a GFA file rather than a VG file but this may have implications for mapping reads.
There’s also an [autoindex](https://github.com/vgteam/vg/wiki/Automatic-indexing-for-read-mapping-and-downstream-inference) command.

In [ ]:
!vg convert -f yprp.chrVIII.pggb.gfa > yprp.chrVIII.pggb.vg

<div class="alert alert-block alert-info"> <b>Try this for the full graph (yprp.fullgenome.pggb.gfa):
</b>  
    <ul>
        <li>Create a blank code cell below.</li>
        <li>Convert yprp.fullgenome.pggb.gfa from GFA format to VG, calling the result "yprp.fullgenome.pggb.vg"</li></a>. </div>
    </ul>


<details>
<summary>Click for help</summary>
<br>
!vg convert -f yprp.fullgenome.pggb.gfa > yprp.fullgenome.pggb.vg
</details>

Now practice on the 16 chromosome subgraphs. Convert each one to vg format. You could do each one individually but a for loop will make it easier. The gfa files are labelled 0-15.

In [ ]:
!for i in {0..15}; \
do \
    vg convert -f output_allchrs/*/*community.${i}.fa*gfa > yprp.allchrs.${i}.vg; \
done

At this point, you can continue to keep them separate but let's merge them into a single graph using 'vg combine'.

In [ ]:
!vg combine yprp.allchrs.{0..15}.vg > yprp.allchrs.pggb.vg

### Indexing with VG

We need to index the graph before we do read mapping. In this case, we will create the necessary indexes for `vg Giraffe`, which we will use for read mapping. We will index with `vg autoindex`.

The parameters:

--workflow  The name of the downstream workflow you are preparing for    
-g  Name of the graph  
-p  Output prefix

Index the chrVII graph.

In [ ]:
!vg autoindex --workflow giraffe -g yprp.chrVIII.pggb.vg -p yprp.chrVIII.pggb

This will create all the necessary files for `vg giraffe`:

yprp.chrVIII.pggb.giraffe.gbz  
+ This ia a [GBZ](https://academic.oup.com/bioinformatics/article/38/22/5012/6731924?login=false) format graph that includes a [GBWT](https://github.com/jltsiren/gbwt) index and the corresponding GBWTGraph.

yprp.chrVIII.pggb.min
+ Minimizer index annotated with positions in the distance index.

yprp.chrVIII.pggb.dist
+ Minimum distance index.

Make sure the files are all there by listing the files.

The parameters:
-l  List it in long form (this will allow us to see the size of the file).

NOTE: This will list some of our original graphs as well.

In [ ]:
!ls -l yprp.chrVIII.pggb.*

<div class="alert alert-block alert-info"> <b>Try this:
</b>  
    <ul>
        <li>Create a blank code cell below.</li>
        <li>Index the graph we made from the full genome (yprp.fullgenome.pggb.vg).</li>        
    </ul>

<details>
<summary>Click for help</summary>
<br>

!vg autoindex --workflow giraffe -g yprp.fullgenome.pggb.vg -p yprp.fullgenome.pggb

</details>

<div class="alert alert-block alert-info"> <b>Try this:
    </b>  
    <ul>
        <li>Create a blank code cell below.</li>
        <li>Index the combined graph, originally run as individual chromosomes (yprp.allchrs.pggb.vg).</li>        
    </ul>

<details>
<summary>Click for help</summary>
<br>

!vg autoindex --workflow giraffe -g yprp.allchrs.pggb.vg -p yprp.allchrs.pggb

</details>

### Graph statistics

Let's get some graph statistics for all 3 graphs, starting with the chrVIII graph.

**vg stats**

+ -z, --size             size of graph
+ -N, --node-count       number of nodes in graph
+ -E, --edge-count       number of edges in graph
+ -l, --length           length of sequences in graph

NOTE: the number of nodes and edges is given by default but calling them explicitly means they will be labeled.

In [ ]:
!vg stats -z -N -E -l yprp.chrVIII.pggb.vg

The length of chromosome VIII is ~5.5-5.8k. So a size of just over 600k makes sense--it covers the full chromosome length plus a little more to account for regions that have diverged between our 3 accessions and, therefore, have more than one version in the graph.

<div class="alert alert-block alert-info"> <b>Try this:</b>  
    <ul>
        <li>Create a blank code cell below.</li>
        <li>Get stats for the graph created directly from the full genome (yprp.fullgenome.pggb.vg).</li>
        <li>Get stats for the graph created by combining the 16 chromosomal subgraphs (yprp.allchrs.pggb.vg).</li>
        <li>What differences do you notice between the stats for these two graphs?</li> 
    </ul>

<details>
<summary>Click for help</summary>
<br>

!vg stats -z -N -E -l yprp.fullgenome.pggb.vg

!vg stats -z -N -E -l yprp.allchrs.pggb.vg

The graph made directly from the full genome has more nodes and edges. This likely reflects the extra connections between chromosomes that you would expect in this graph, made with all the chromosomes at once, that are not in the other graph, where chromosomes were isolated during the building of the graph. These extra connections may have created additional nodes when they connected not quite identical sequence between chromosomes, splitting what would have been a single node of identical sequence into more than one node to take account of the sequence differences.

The graph made from individual chromosome subgraphs has a slightly longer sequence length, possibly reflecting some duplication between chromosomes that wasn't fully merged.

</details>

## Conclusion
In this submodule you learned about the VG toolkit and what it can be used for. You also learned how to index graphs with VG and indexed the PGGB graphs that you had previously made. You also learned how to obtain graph statistics. In the next submodule, you will learn how to map reads to the indexed graph.

## Clean up
No cleanup is necessary for this submodule. Don't forget to shutdown your Workbench when you are done working through this module!